# EWX Short List

Want to look at meters in EWX short list to see if we can verify forecasted & actual statistics / data integrity.

In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import json
import time
import math
import os
import cx_Oracle
import pprint
pp = pprint.PrettyPrinter(3)

In [2]:
base = os.getcwd()
print('working in {}.'.format(base))

working in C:\Users\wb5888\LA-tools.


## Query PE

In [16]:
def OracleAPI(query):
    
    uid = 'tesi_interface'
    pwd = 'peint88'

    ip = '172.25.152.125'
    port = '1700'
    service_name = 'tppe.mytna.com'
    dsn = cx_Oracle.makedsn(ip, port, service_name=service_name)
    
    result_list = []
    con = cx_Oracle.connect(user = uid, password = pwd, dsn = dsn)
    cur = con.cursor()
    cur.execute(query)
    
    for result in cur:
        result_list.append(result)
        
    return(result_list)

imported CapTags successfully.


In [ ]:
def pipe_import(filename, path):

    account = filename.split('.')[0]
    ch = '3'
    final = ",".join([account, ch])
    write_path = os.path.join(path,filename)

    cmd_prmpt = ["C:\LODESTAR\Bin\intdexp", "-c", "Data Source=TPPE;User ID=tesi_interface;Password=peint88;LSProvider=ODP;",\
                 "-q", "pwrline", "-f", "C:\LODESTAR\cfg\examples\Cfg\lodestar.cfg", "-s", "01/01/2019", "-t", "12/31/2019",\
                 "-dtuse", "PARTIAL", "-d", "hi", final, "-o", write_path]

    x = Popen(cmd_prmpt, stdout = PIPE, stderr = PIPE)
    output, errors = x.communicate()

## Read Data

In [3]:
ewx = pd.read_csv('short_lists_ewx_top_ten.csv', header = 0)
ewx['AGREE_START_DATE'] = pd.to_datetime(ewx['AGREE_START_DATE'])
ewx.head()

,AGREEMENT,QUOTENUMBER,REVISION,METERID,AGREE_START_DATE,PR_RELEASED_DATE,MAXPARSEDUIDRAW,RECENT_PARSED_FOR_PR,SUM_OF_USAGE,ALPS_ANNUAL_KWH,...,ANNUALIZED_ALPS_ANNUAL_KWH,EWN_IN_OUT_DIFF,EWX_OUT_VS_IRIS_DIFF,IRIS_VS_RAW_DIFF,EWX_sent_timestamp,PE_LSTIME,METER_TYPE,CHANNEL,BROKER_NAME,Unnamed: 22
0,1-HFQXTB,1-HF3YIL,2,ERCOT_CPT_1008901006901276940117,1970-01-01 00:00:00.000043405,43276.66847,2593481,43276.61031,13.08605_12,1.308605e+01,...,1.308605e+01,2.888953e+06,0.025421,2.815514e+06,06/25/2018 14:42:00,07/03/2018 23:51:38,SCALAR,Broker,NORESCO LLC,NaN
1,1-HFQXTB,1-HF3YIL,2,ERCOT_CPT_1008901024901276770117,1970-01-01 00:00:00.000043405,43276.66847,2593478,43276.61030,13.08605_12,1.308605e+01,...,1.308605e+01,1.444514e+06,0.025446,1.407757e+06,06/25/2018 14:42:00,07/19/2018 16:22:36,SCALAR,Broker,NORESCO LLC,NaN
2,1-GXF0NO,1-GPY4BN,5,NEPOOL_UI_1140038927020,1970-01-01 00:00:00.000043221,43179.58803,2290943,43144.44807,289855.8621_12,2.898559e+05,...,2.898559e+05,2.855124e+01,0.068458,2.652823e+01,03/15/2018 07:21:00,08/19/2018 23:31:42,SCALAR,KPG Broker,ENERNOC INC,NaN
3,1-I4ZI49,1-I40NDU,1,PJM_PPL_6102785013,1970-01-01 00:00:00.000043466,43444.38110,2713245,43356.42718,642177.3399_12,6.421773e+05,...,6.421773e+05,7.049023e+00,0.004719,7.011041e+00,09/13/2018 10:17:00,12/03/2018 09:37:18,SCALAR,Direct,0,NaN
4,1-HH450N,1-GV0UPL,1,NEPOOL_CLP_51768353054_427832005,1970-01-01 00:00:00.000043282,43174.69051,2347285,43172.73895,3563508.9401_12,3.563509e+06,...,3.563509e+06,7.037864e+00,0.875270,2.561928e-03,03/15/2018 09:21:00,07/03/2018 16:34:24,IDR,KPG Broker,TFS- ENERGY SOLUTIONS LP,NaN


In [8]:
meters = list(ewx.METERID)
print('found {} meters, data struct: {}.'.format(len(meters), type(meters)))

found 39 meters, data struct: <class 'list'>.


In [34]:
a = meters[0]
a

'ERCOT_CPT_1008901006901276940117'

## Pipe Import

In [ ]:
''' pipe_import '''

''' takes inputs of:
        filename (str) i.e. 'meter_id.csv'
        ch (int)       i.e. 1
        start (str)    i.e. 'mm/dd/YYYY' as a character string, NOT datetime, pytz, timestamp objects!!
        stop (str)    i.e. 'mm/dd/YYYY' as a character string, NOT datetime, pytz, timestamp objects!!
        path (str)     i.e. 'C://Users/wb5888/Documents/IDR'  
'''  

''' imports ch for meter included in filename, "meter_id.csv" '''
''' between start and stop dates. Writes results to path. '''

In [119]:
def pipe_import(filename, ch, start, stop, path):
    
    account = filename.split('.')[0]
    channel = str(ch)
    final = ",".join([account, str(ch)])
    write_path = os.path.join(path,filename)
    
    start = '"'.join(['', start, ''])
    stop = '"'.join(['', stop, ''])

    cmd_prmpt = ["C:\LODESTAR\Bin\intdexp", "-c", "Data Source=TPPE;User ID=tesi_interface;Password=peint88;LSProvider=ODP;",\
                "-q", "pwrline", "-f", "C:\LODESTAR\cfg\examples\Cfg\lodestar.cfg", "-s", start, "-t", stop,\
                "-dtuse", "PARTIAL", "-d", "hi", final, "-o", write_path]
    
    print('call .exe with prompt...')
    pp.pprint(cmd_prmpt)
    
    return(ct.issue_command(cmd_prmpt))

## Example

In [120]:
test = '.'.join([a, 'csv'])
print('working with file: {}.'.format(test))
print('')

data = os.path.join(base, 'ewx_short_list_data')
print('writing data to: {}.'.format(data))

working with file: ERCOT_CPT_1008901006901276940117.csv.

writing data to: C:\Users\wb5888\LA-tools\ewx_short_list_data.


In [132]:
output, error = pipe_import(test, 1, "01/01/2017", "11/01/2017", data)

call .exe with prompt...
[  'C:\\LODESTAR\\Bin\\intdexp',
   '-c',
   'Data Source=TPPE;User ID=tesi_interface;Password=peint88;LSProvider=ODP;',
   '-q',
   'pwrline',
   '-f',
   'C:\\LODESTAR\\cfg\\examples\\Cfg\\lodestar.cfg',
   '-s',
   '"01/01/2017"',
   '-t',
   '"11/01/2017"',
   '-dtuse',
   'PARTIAL',
   '-d',
   'hi',
   'ERCOT_CPT_1008901006901276940117,1',
   '-o',
   'C:\\Users\\wb5888\\LA-tools\\ewx_short_list_data\\ERCOT_CPT_1008901006901276940117.csv']


In [133]:
print((output, error))

(b'Interval Data Export terminated with error.\r\n', b'')


In [122]:
files = os.listdir(data)
files

['ERCOT_CPT_1008901006901276940117.csv']

## Read data from disk

In [123]:
def data_import(file, skip):

    print('reading data...')
    data = pd.read_csv(file, header = None, skiprows = skip, index_col = None, sep = ",")

    print(data.head())
    print('reindexing...')
    data.reset_index(drop = True, inplace = True)
    data.drop(data.columns[[1, 3]], axis = 1, inplace = True)
    
    print('type conversion...')
    data.columns = ['v', 't']
    data['t'] = pd.to_datetime(data['t'])
    data.index = data['t']
    data.drop(data.columns[1], axis = 1, inplace = True)
    
    print('write to csv...')
    data.to_csv(file, header = False, index = False)
    print('done.')
    
    return(data)

def ten_yr_skip():
    reads = [8760, 8784, 8760, 8760, 8760, 8784, 8760, 8760, 8760, 8784, 8760]
    skip_index = list(range(0,5))
    cum_x = 0

    for i, x in enumerate(reads):
        cum_x += x
        cur_index = list(range(0 + cum_x + (i + 1)*5, 5 + cum_x + (i + 1)*5))
        for index in cur_index:
            skip_index.append(index)
            
    return(skip_index)

## Example...

In [126]:
test = files[0]

skip_index = ten_yr_skip()
print('skip_index attributes:')
print('(len, type, head): {}.'.format((len(skip_index), type(skip_index), skip_index[:5])))
print('')
print('working with file: {}.'.format(test))

skip_index attributes:
(len, type, head): (60, <class 'list'>, [0, 1, 2, 3, 4]).

working with file: ERCOT_CPT_1008901006901276940117.csv.


In [127]:
os.chdir(data)
meter_data = data_import(test, skip_index)

reading data...
          0
0  2165.768
1  2407.184
2  2474.244
3  3922.744
4  5290.772
reindexing...


IndexError: index 1 is out of bounds for axis 0 with size 1

### Show Data

In [98]:
meter_data.head()

,v
t,
2016-11-21 00:00:00,2434.008
2016-11-21 01:00:00,2004.824
2016-11-21 02:00:00,1749.996
2016-11-21 03:00:00,1589.052
2016-11-21 04:00:00,1669.524


## Statistics

In [101]:
total_use = meter_data.apply(sum, axis = 0)
days = dt.timedelta(max(meter_data.index) - min(meter_data.index))

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [105]:
meter_data.index.to_series()

pandas.core.series.Series